This notebook contains the different steps needed to link the biodiversity datacubes to the half earth goal. See Jira subtasks in issue [HE-11](https://vizzuality.atlassian.net/browse/HE-11?atlOrigin=eyJpIjoiZDk3ZDdmNDZmMTZhNDJmMzk1ZGFhNWY3ZDA2Njc0NDMiLCJwIjoiaiJ9). 
Make sure the parallel processing is set to at least `80%`. 

# get range of each species

In [ ]:
# takes 16 minutes when parallel processing is not set up
arcpy.sa.ZonalStatisticsAsTable("americas_rectangle", 
                                "OBJECTID", 
                                "hummingbirds_binary.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\AOIs_development\AOIs_development.gdb\all_stats_americas", 
                                "DATA", "ALL", "ALL_SLICES", 90)

In [ ]:
arcpy.management.JoinField("hummingbirds_binary_lookup", 
                           "SliceNumber", 
                           "all_stats_americas", 
                           "SliceNumber", 
                           "SUM")

In [ ]:
arcpy.management.AlterField("hummingbirds_binary_lookup", 
                            "SUM", "range_size", 
                            '', "DOUBLE", 8, "NULLABLE", 
                            "DO_NOT_CLEAR")

# overlap protection and obtain how much (proportion) of the range of the species is protected

## Preparing WDPA data
WDPA User Manual [url](https://wdpa.s3-eu-west-1.amazonaws.com/WDPA_Manual/English/WDPA_WDOECM_Manual_1_6.pdf) and excerpts of MOL pre prints explaining the cleaning process: 
>  We excluded PAs that did not have designated, inscribed, or
established status, points without a reported area, terrestrial reserves, and UNESCO Man and Biosphere Reserves. The remaining points were buffered to a size equal to the reported area, combined with the polygons and dissolved to create a global MPA layer. This layer was then rasterized to the same resolution and extent as the human modification data, and used to remove any HM 1-km pixels located inside MPAs so that currently protected areas did not contribute to aggregated summaries of HM.

Rinnan et al. 2021 ([preprint](https://www.biorxiv.org/content/10.1101/2021.04.23.441004v1.full.pdf))

> we followed the WDPA’s recommendations on cleaning data for calculations of global coverage: we excluded PAs that did not have designated, inscribed, or established status, points without a reported area, marine reserves, and UNESCO Man and Biosphere Reserves.

Rinnan and Jetz 2020 ([preprint](https://www.biorxiv.org/content/10.1101/2020.02.05.936047v1.full.pdf))

### exclude PAs without designated, inscribed or established status + points without a reported area and unesco man and biosphere reserves. 
Do this in both [points](https://services5.arcgis.com/Mj0hjvkNtV7NRhA7/arcgis/rest/services/WDPA_v0/FeatureServer) and [polygons](https://services5.arcgis.com/Mj0hjvkNtV7NRhA7/arcgis/rest/services/WDPA_v0/FeatureServer) of WDPA

#### Points
The query for points include filtering by reported area. 

In [ ]:
arcpy.management.SelectLayerByAttribute(r"WDPA - World Database of Protected Areas\WDPA_point_Latest", 
                                        "NEW_SELECTION", 
                                        "STATUS <> 'Not Reported' And STATUS <> 'Proposed' And REP_AREA IS NOT NULL And REP_AREA > 0 And DESIG_ENG <> 'UNESCO-MAB Biosphere Reserve'", 
                                        None)

In [ ]:
arcpy.conversion.FeatureClassToFeatureClass(r"WDPA - World Database of Protected Areas\WDPA_point_Latest", 
                                            r"C:\Users\Viz1\Documents\ArcGIS\Projects\AOIs_development\AOIs_development.gdb", 
                                            "queried_wdpa_points", '', 
                                            r'WDPAID "WDPAID" true true false 0 Double 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,WDPAID,-1,-1;WDPA_PID "Parcel ID" true true false 52 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,WDPA_PID,0,52;PA_DEF "Protected Area Definition" true true false 20 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,PA_DEF,0,20;NAME "Name" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,NAME,0,254;ORIG_NAME "Original Name" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,ORIG_NAME,0,254;DESIG "Designation" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,DESIG,0,254;DESIG_ENG "Designation English" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,DESIG_ENG,0,254;DESIG_TYPE "Designation Type" true true false 20 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,DESIG_TYPE,0,20;IUCN_CAT "IUCN Management Category" true true false 20 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,IUCN_CAT,0,20;INT_CRIT "International Criteria" true true false 100 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,INT_CRIT,0,100;MARINE "Marine" true true false 20 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,MARINE,0,20;REP_M_AREA "Reported Marine Area (km2)" true true false 0 Double 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,REP_M_AREA,-1,-1;REP_AREA "Reported Area (km2)" true true false 0 Double 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,REP_AREA,-1,-1;NO_TAKE "No-take" true true false 50 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,NO_TAKE,0,50;NO_TK_AREA "No-take Area (km2)" true true false 0 Double 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,NO_TK_AREA,-1,-1;STATUS "Status" true true false 100 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,STATUS,0,100;STATUS_YR "Status Year" true true false 0 Long 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,STATUS_YR,-1,-1;GOV_TYPE "Governance Type" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,GOV_TYPE,0,254;OWN_TYPE "Ownership Type" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,OWN_TYPE,0,254;MANG_AUTH "Management Authority" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,MANG_AUTH,0,254;MANG_PLAN "Management Plan" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,MANG_PLAN,0,254;VERIF "Verification" true true false 20 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,VERIF,0,20;METADATAID "Metadata ID" true true false 0 Long 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,METADATAID,-1,-1;SUB_LOC "Sub-national Location" true true false 100 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,SUB_LOC,0,100;PARENT_ISO3 "Parent ISO3 Code" true true false 50 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,PARENT_ISO3,0,50;ISO3 "ISO3 Code" true true false 50 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_point_Latest,ISO3,0,50', '')

#### Polygons
The query for polygons does not include filtering by reported area. 

In [ ]:
arcpy.management.SelectLayerByAttribute(r"WDPA - World Database of Protected Areas\WDPA_poly_Latest", 
                                        "NEW_SELECTION", 
                                        "STATUS <> 'Adopted' And STATUS <> 'Proposed' And DESIG_ENG <> 'UNESCO-MAB Biosphere Reserve'", 
                                        None)

In [ ]:
# this took 18 minutes... 
arcpy.conversion.FeatureClassToFeatureClass(r"WDPA - World Database of Protected Areas\WDPA_poly_Latest", 
                                            r"C:\Users\Viz1\Documents\ArcGIS\Projects\AOIs_development\AOIs_development.gdb", "queried_wdpa_polys", '', r'WDPAID "WDPAID" true true false 0 Double 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,WDPAID,-1,-1;WDPA_PID "Parcel ID" true true false 50 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,WDPA_PID,0,50;PA_DEF "Protected Area Definition" true true false 20 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,PA_DEF,0,20;NAME "Name" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,NAME,0,254;ORIG_NAME "Original Name" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,ORIG_NAME,0,254;DESIG "Designation" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,DESIG,0,254;DESIG_ENG "English Designation" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,DESIG_ENG,0,254;DESIG_TYPE "Designation Type" true true false 20 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,DESIG_TYPE,0,20;IUCN_CAT "IUCN Management Category" true true false 20 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,IUCN_CAT,0,20;INT_CRIT "International Criteria" true true false 100 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,INT_CRIT,0,100;MARINE "Marine" true true false 20 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,MARINE,0,20;REP_M_AREA "Reported Marine Area (km2)" true true false 0 Double 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,REP_M_AREA,-1,-1;GIS_M_AREA "GIS Marine Area (km2)" true true false 0 Double 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,GIS_M_AREA,-1,-1;REP_AREA "Reported Area (km2)" true true false 0 Double 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,REP_AREA,-1,-1;GIS_AREA "GIS Area (km2)" true true false 0 Double 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,GIS_AREA,-1,-1;NO_TAKE "No-take" true true false 50 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,NO_TAKE,0,50;NO_TK_AREA "No-take Area (km2)" true true false 0 Double 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,NO_TK_AREA,-1,-1;STATUS "Status" true true false 100 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,STATUS,0,100;STATUS_YR "Status Year" true true false 0 Long 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,STATUS_YR,-1,-1;GOV_TYPE "Governance Type" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,GOV_TYPE,0,254;OWN_TYPE "Ownership Type" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,OWN_TYPE,0,254;MANG_AUTH "Management Authority" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,MANG_AUTH,0,254;MANG_PLAN "Management Plan" true true false 254 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,MANG_PLAN,0,254;VERIF "Verification" true true false 20 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,VERIF,0,20;METADATAID "Metadata ID" true true false 0 Long 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,METADATAID,-1,-1;SUB_LOC "Sub-national Location" true true false 100 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,SUB_LOC,0,100;PARENT_ISO3 "Parent ISO3 Code" true true false 50 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,PARENT_ISO3,0,50;ISO3 "ISO3 Code" true true false 50 Text 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,ISO3,0,50;Shape__Area "Shape__Area" false true false 0 Double 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,Shape__Area,-1,-1;Shape__Length "Shape__Length" false true false 0 Double 0 0,First,#,WDPA - World Database of Protected Areas\WDPA_poly_Latest,Shape__Length,-1,-1', '')

### make buffered polygons from points using reported area
The formula to calculate an area: `A = pi * r^2`. The radius of the buffer should be equal to: `sqrt(REP_AREA/pi)`. the buffering must be done in a projected spatial reference. `REP_AREA` is the reported area in km2. 

In [ ]:
arcpy.management.Project("queried_wdpa_points", 
                         r"C:\Users\Viz1\Documents\ArcGIS\Projects\AOIs_development\AOIs_development.gdb\queried_wdpa_points_Behrmann", 
                         "PROJCS['World_Behrmann',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Behrmann'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],UNIT['Meter',1.0]]", 
                         None, 
                         "PROJCS['WGS_1984_Web_Mercator_Auxiliary_Sphere',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Mercator_Auxiliary_Sphere'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],PARAMETER['Standard_Parallel_1',0.0],PARAMETER['Auxiliary_Sphere_Type',0.0],UNIT['Meter',1.0]]", 
                         "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

In [ ]:
arcpy.management.CalculateField("queried_wdpa_points_Behrmann", 
                                "buffer_radius", 
                                "math.sqrt(!REP_AREA!/math.pi)", 
                                "PYTHON3", '', "TEXT")

In [ ]:
arcpy.management.CalculateField("queried_wdpa_points_Behrmann", 
                                "buffer_radius_unit", 
                                'f"{!buffer_radius!} Kilometers"', 
                                "PYTHON3", '', "TEXT")

In [ ]:
arcpy.analysis.Buffer("queried_wdpa_points_Behrmann", 
                      r"C:\Users\Viz1\Documents\ArcGIS\Projects\AOIs_development\AOIs_development.gdb\queried_wdpa_points_buffer", 
                      "buffer_radius_unit",
                      "FULL", "ROUND", "NONE", None, "PLANAR")

### Merging the polygons and the buffered points. 
Put both feature classes in the same spatial coordinate system. 

In [ ]:
arcpy.management.Project("queried_wdpa_points_buffer", 
                         r"C:\Users\Viz1\Documents\ArcGIS\Projects\AOIs_development\AOIs_development.gdb\queried_wdpa_points_buffer_mercator", 
                         "PROJCS['WGS_1984_Web_Mercator_Auxiliary_Sphere',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Mercator_Auxiliary_Sphere'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],PARAMETER['Standard_Parallel_1',0.0],PARAMETER['Auxiliary_Sphere_Type',0.0],UNIT['Meter',1.0]]", 
                         None, 
                         "PROJCS['WGS_1984_Web_Mercator_Auxiliary_Sphere',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Mercator_Auxiliary_Sphere'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],PARAMETER['Standard_Parallel_1',0.0],PARAMETER['Auxiliary_Sphere_Type',0.0],UNIT['Meter',1.0]]", 
                         "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

In [ ]:
arcpy.management.Merge("queried_wdpa_polys;queried_wdpa_points_buffer_mercator", 
                       r"C:\Users\Viz1\Documents\ArcGIS\Projects\AOIs_development\AOIs_development.gdb\clean_wdpa_Merge", 
                       'WDPAID "WDPAID" true true false 8 Double 0 0,First,#,queried_wdpa_polys,WDPAID,-1,-1,queried_wdpa_points_buffer_mercator,WDPAID,-1,-1;WDPA_PID "Parcel ID" true true false 50 Text 0 0,First,#,queried_wdpa_polys,WDPA_PID,0,50,queried_wdpa_points_buffer_mercator,WDPA_PID,0,52;PA_DEF "Protected Area Definition" true true false 20 Text 0 0,First,#,queried_wdpa_polys,PA_DEF,0,20,queried_wdpa_points_buffer_mercator,PA_DEF,0,20;NAME "Name" true true false 254 Text 0 0,First,#,queried_wdpa_polys,NAME,0,254,queried_wdpa_points_buffer_mercator,NAME,0,254;ORIG_NAME "Original Name" true true false 254 Text 0 0,First,#,queried_wdpa_polys,ORIG_NAME,0,254,queried_wdpa_points_buffer_mercator,ORIG_NAME,0,254;DESIG "Designation" true true false 254 Text 0 0,First,#,queried_wdpa_polys,DESIG,0,254,queried_wdpa_points_buffer_mercator,DESIG,0,254;DESIG_ENG "English Designation" true true false 254 Text 0 0,First,#,queried_wdpa_polys,DESIG_ENG,0,254,queried_wdpa_points_buffer_mercator,DESIG_ENG,0,254;DESIG_TYPE "Designation Type" true true false 20 Text 0 0,First,#,queried_wdpa_polys,DESIG_TYPE,0,20,queried_wdpa_points_buffer_mercator,DESIG_TYPE,0,20;IUCN_CAT "IUCN Management Category" true true false 20 Text 0 0,First,#,queried_wdpa_polys,IUCN_CAT,0,20,queried_wdpa_points_buffer_mercator,IUCN_CAT,0,20;INT_CRIT "International Criteria" true true false 100 Text 0 0,First,#,queried_wdpa_polys,INT_CRIT,0,100,queried_wdpa_points_buffer_mercator,INT_CRIT,0,100;MARINE "Marine" true true false 20 Text 0 0,First,#,queried_wdpa_polys,MARINE,0,20,queried_wdpa_points_buffer_mercator,MARINE,0,20;REP_M_AREA "Reported Marine Area (km2)" true true false 8 Double 0 0,First,#,queried_wdpa_polys,REP_M_AREA,-1,-1,queried_wdpa_points_buffer_mercator,REP_M_AREA,-1,-1;GIS_M_AREA "GIS Marine Area (km2)" true true false 8 Double 0 0,First,#,queried_wdpa_polys,GIS_M_AREA,-1,-1;REP_AREA "Reported Area (km2)" true true false 8 Double 0 0,First,#,queried_wdpa_polys,REP_AREA,-1,-1,queried_wdpa_points_buffer_mercator,REP_AREA,-1,-1;GIS_AREA "GIS Area (km2)" true true false 8 Double 0 0,First,#,queried_wdpa_polys,GIS_AREA,-1,-1;NO_TAKE "No-take" true true false 50 Text 0 0,First,#,queried_wdpa_polys,NO_TAKE,0,50,queried_wdpa_points_buffer_mercator,NO_TAKE,0,50;NO_TK_AREA "No-take Area (km2)" true true false 8 Double 0 0,First,#,queried_wdpa_polys,NO_TK_AREA,-1,-1,queried_wdpa_points_buffer_mercator,NO_TK_AREA,-1,-1;STATUS "Status" true true false 100 Text 0 0,First,#,queried_wdpa_polys,STATUS,0,100,queried_wdpa_points_buffer_mercator,STATUS,0,100;STATUS_YR "Status Year" true true false 4 Long 0 0,First,#,queried_wdpa_polys,STATUS_YR,-1,-1,queried_wdpa_points_buffer_mercator,STATUS_YR,-1,-1;GOV_TYPE "Governance Type" true true false 254 Text 0 0,First,#,queried_wdpa_polys,GOV_TYPE,0,254,queried_wdpa_points_buffer_mercator,GOV_TYPE,0,254;OWN_TYPE "Ownership Type" true true false 254 Text 0 0,First,#,queried_wdpa_polys,OWN_TYPE,0,254,queried_wdpa_points_buffer_mercator,OWN_TYPE,0,254;MANG_AUTH "Management Authority" true true false 254 Text 0 0,First,#,queried_wdpa_polys,MANG_AUTH,0,254,queried_wdpa_points_buffer_mercator,MANG_AUTH,0,254;MANG_PLAN "Management Plan" true true false 254 Text 0 0,First,#,queried_wdpa_polys,MANG_PLAN,0,254,queried_wdpa_points_buffer_mercator,MANG_PLAN,0,254;VERIF "Verification" true true false 20 Text 0 0,First,#,queried_wdpa_polys,VERIF,0,20,queried_wdpa_points_buffer_mercator,VERIF,0,20;METADATAID "Metadata ID" true true false 4 Long 0 0,First,#,queried_wdpa_polys,METADATAID,-1,-1,queried_wdpa_points_buffer_mercator,METADATAID,-1,-1;SUB_LOC "Sub-national Location" true true false 100 Text 0 0,First,#,queried_wdpa_polys,SUB_LOC,0,100,queried_wdpa_points_buffer_mercator,SUB_LOC,0,100;PARENT_ISO3 "Parent ISO3 Code" true true false 50 Text 0 0,First,#,queried_wdpa_polys,PARENT_ISO3,0,50,queried_wdpa_points_buffer_mercator,PARENT_ISO3,0,50;ISO3 "ISO3 Code" true true false 50 Text 0 0,First,#,queried_wdpa_polys,ISO3,0,50,queried_wdpa_points_buffer_mercator,ISO3,0,50;Shape__Area "Shape__Area" true true false 8 Double 0 0,First,#,queried_wdpa_polys,Shape__Area,-1,-1;Shape__Length "Shape__Length" true true false 8 Double 0 0,First,#,queried_wdpa_polys,Shape__Length,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,queried_wdpa_polys,Shape_Length,-1,-1,queried_wdpa_points_buffer_mercator,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,queried_wdpa_polys,Shape_Area,-1,-1,queried_wdpa_points_buffer_mercator,Shape_Area,-1,-1;buffer_radius "buffer_radius" true true false 8 Double 0 0,First,#,queried_wdpa_points_buffer_mercator,buffer_radius,-1,-1;buffer_radius_unit "buffer_radius_unit" true true false 255 Text 0 0,First,#,queried_wdpa_points_buffer_mercator,buffer_radius_unit,0,255;BUFF_DIST "BUFF_DIST" true true false 8 Double 0 0,First,#,queried_wdpa_points_buffer_mercator,BUFF_DIST,-1,-1;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,queried_wdpa_points_buffer_mercator,ORIG_FID,-1,-1', "NO_SOURCE_INFO")

**This resulting layer could be used as the protected areas and have tooltips.** To have community and non community the protocol to follow is described in [WDPA subset and merging with RAISG](https://github.com/Vizzuality/he-scratchfolder/blob/master/arcpyNotebooks/WDPA%20subset%20and%20merging%20with%20RAISG.ipynb).

### Rasterizing
Give a constant value to all the features so that is the value of the raster. Dissolving might be needed first. Dissolving is a nightmare... 

In [ ]:
arcpy.management.CalculateField("clean_wdpa_Merge", "buffer_radius", "1", "PYTHON3", '', "TEXT")

In [ ]:
# this took two minutes
arcpy.conversion.PolygonToRaster("clean_wdpa_Merge", 
                                 "buffer_radius", 
                                 r"C:\Users\Viz1\Documents\ArcGIS\Projects\AOIs_development\AOIs_development.gdb\clean_wdpa_raster", 
                                 "CELL_CENTER", "NONE", 
                                 r"C:\Users\Viz1\Documents\ArcGIS\Projects\Sample_testing\yaleCube.acs\hummingbirds_binary.crf")

## identifying overlap between species range and protected areas


In [ ]:
# again, i forgot the parallel processing... It took 1h 7min.
arcpy.ia.ZonalStatisticsAsTable("clean_wdpa_raster", 
                                "Value", 
                                "hummingbirds_binary.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\AOIs_development\AOIs_development.gdb\all_stats_wdpa", 
                                "DATA", "ALL", "ALL_SLICES", 90)

In [ ]:
arcpy.management.JoinField("hummingbirds_binary_lookup", 
                           "SliceNumber", 
                           "all_stats_wdpa", 
                           "SliceNumber", 
                           "SUM")

In [ ]:
arcpy.management.AlterField("hummingbirds_binary_lookup", 
                            "SUM", 
                            "protected_range_size", 
                            '', 
                            "DOUBLE", 
                            8, 
                            "NULLABLE", 
                            "DO_NOT_CLEAR")

In [2]:
arcpy.management.CalculateField("hummingbirds_binary_lookup", 
                                "protection_target", 
                                "calculateTarget(!range_size!)", 
                                "PYTHON3", 
                                r"import numpy
def calculateTarget(sp_range):
    if (sp_range <= 10000):
        target = 100
        #return target
    elif (sp_range > 250000):
        target = 15
        #return target
    else:
        target = numpy.log(sp_range)
    return target", 
                                "TEXT")